# Labolatorium 4 - Odległość edycyjna

In [1]:
import numpy as np
from spacy.tokens import Token
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

## Łańcuchy tekstu

In [2]:
strings1 = ["los", "kloc"]
strings2 = ["Łódź", "Lodz"]
strings3 = ["kwintesencja", "quintessence"]
strings4 = ["ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"]

## Funkcja obliczająca odległość edycyjną

Funkcja obliczająca odległość edycyjną, zwraca tablice zawierającą na indeksie i, j odległość edycyjną i liter tekstu 1 i j liter tekstu 2 oraz informację w jaki sposób ją osiągnęliśmy w tym kroku: 1 wartość po lewej + 1, 2 wartość z góry + 1, 3 wartość po skosie + oblicczona delta

In [3]:
def simple_delta (x, y):
    if x == y:
        return 0
    return 1

#1 <- (left), 2 <| (up), 3 <\ (cross)
def levenshtein_distance (text1, text2, delta = simple_delta):
    len1 = len(text1)
    len2 = len(text2)
    arr = np.zeros((len1+1, len2+1, 2))
    for i in range(1, len1+1):
        arr[i, 0, 0] = i
        arr[i, 0, 1] = 1
    for i in range(1, len2+1):
        arr[0, i, 0] = i
        arr[0, i, 1] = 2
    for i in range (1, len1+1):
        for j in range (1, len2+1):
            if arr[i-1, j, 0] < arr[i, j-1, 0]:
                step = 1
                val = arr[i-1, j, 0] + 1
            else:
                step = 2
                val = arr[i, j-1, 0] + 1
            d = delta (text1[i-1], text2[j-1]) + arr[i-1, j-1, 0]
            if d < val:
                val = d
                step = 3
            arr[i, j, 0] = val
            arr[i, j, 1] = step
    return arr

## Wizualizacja

Oblicza tablicę za pomocą powyższej funkcji jeśli jej nie dostanie. Na podstawie drugiej wartości w krotce w tablicy wylicza ścieżkę otrzymania tekstu2 (jedną z możliwych dla danej odległości edycyjnej). Tworzy wizualizację tekstową z opisem.

In [4]:
def visualization (text1, text2, arr = None, delta = simple_delta):
    if arr == None:
        arr = levenshtein_distance (text1, text2, delta)
    print ("Odległość edycyjna wynosi", arr[-1, -1, 0])
    path = []
    i = len (text1)
    j = len (text2)
    while True:
        v = arr[i, j, 1]
        path.append(v)
        if v == 1:
            i -= 1
        elif v == 2:
            j -= 1
        elif v == 3:
            i -= 1
            j -= 1
        else:
            break
    
    path.reverse()   
    i = 0
    j = 0
    text = text1
    print (text)
    for step in path:
        if step == 1:
            print (text[:i]+'*'+text[i]+'*'+text[i+1:], "-> Usuwamy literkę", text[i])
            text = text[:i]+text[i+1:]
            print (text)
        elif step == 2:
            print (text[:i]+'*'+text2[j]+'*'+text[i:], "-> Wstawiamy literkę", text2[j])
            text = text[:i]+text2[j]+text[i:]
            print (text)
            i += 1
            j += 1
        elif step == 3:
            if text[i] != text2 [j]:
                print (text[:i]+'*'+text2[j]+'*'+text[i+1:], "-> Zamiana literki", text[i], "na", text2[j])
                text = text[:i]+text2[j]+text[i+1:]
                print (text)
            i += 1
            j += 1
    if text2 == text:
        print ("Udało się uzyskać słowo:", text2)
    else:
        print ("Błąd")

## Testy

In [5]:
visualization (*strings1)

Odległość edycyjna wynosi 2.0
los
*k*los -> Wstawiamy literkę k
klos
klo*c* -> Zamiana literki s na c
kloc
Udało się uzyskać słowo: kloc


In [6]:
visualization (*strings2)

Odległość edycyjna wynosi 3.0
Łódź
*L*ódź -> Zamiana literki Ł na L
Lódź
L*o*dź -> Zamiana literki ó na o
Lodź
Lod*z* -> Zamiana literki ź na z
Lodz
Udało się uzyskać słowo: Lodz


In [7]:
visualization (*strings3)

Odległość edycyjna wynosi 5.0
kwintesencja
*q*wintesencja -> Zamiana literki k na q
qwintesencja
q*u*intesencja -> Zamiana literki w na u
quintesencja
quintes*s*encja -> Wstawiamy literkę s
quintessencja
quintessenc*e*a -> Zamiana literki j na e
quintessencea
quintessence*a* -> Usuwamy literkę a
quintessence
Udało się uzyskać słowo: quintessence


In [8]:
visualization (*strings4)

Odległość edycyjna wynosi 7.0
ATGAATCTTACCGCCTCG
ATGA*G*TCTTACCGCCTCG -> Zamiana literki A na G
ATGAGTCTTACCGCCTCG
ATGAG*G*CTTACCGCCTCG -> Zamiana literki T na G
ATGAGGCTTACCGCCTCG
ATGAGGCT*C*TACCGCCTCG -> Wstawiamy literkę C
ATGAGGCTCTACCGCCTCG
ATGAGGCTCT*G*CCGCCTCG -> Zamiana literki A na G
ATGAGGCTCTGCCGCCTCG
ATGAGGCTCTG*G*CCGCCTCG -> Wstawiamy literkę G
ATGAGGCTCTGGCCGCCTCG
ATGAGGCTCTGGCC*G*CCTCG -> Usuwamy literkę G
ATGAGGCTCTGGCCCCTCG
ATGAGGCTCTGGCCCCT*C*G -> Usuwamy literkę C
ATGAGGCTCTGGCCCCTG
Udało się uzyskać słowo: ATGAGGCTCTGGCCCCTG


## Najdłuższy wspólny podciąg

Wyszukuje najdłuższy wspólny podciąg korzystając z ogległości edycyjnej poprzez odpowiednią funkcję delta, która nie pozwala na zamianę liter (dla zamiany zwraca koszt 10). Odczytuje ścieżkę powstania zczytując przejścia po skosie (powtarzające się elementy w obu ciągach). 

In [9]:
def lss_delta (x, y):
    if isinstance (x, Token):
        if x.text == y.text:
            return 0
    else:
        if x == y:
            return 0
    return 10

def longest_subsequence (sequence1, sequence2):
    arr = levenshtein_distance (sequence1, sequence2, lss_delta)
    subsequence = []
    i = len (sequence1)
    j = len (sequence2)
    while True:
        v = arr[i, j, 1]
        if v == 1:
            i -= 1
        elif v == 2:
            j -= 1
        elif v == 3:
            i -= 1
            j -= 1
            subsequence.append(sequence1[i])
        else:
            break
    subsequence.reverse()
    return subsequence

## Wczytanie i przygotowanie tekstu

In [10]:
f = open ("romeo-i-julia-700.txt", 'r')
text = f.read ()
f.close ()
#nlp = spacy.load("pl_core_news_sm")
#tokenizer = Tokenizer(nlp.vocab)
#tokens = tokenizer (text)
tokens = text.split("\n")

In [11]:
version1 = []
to_delete = np.random.randint (0, len(tokens), int(0.3*len(tokens)))
for i, token in enumerate(tokens):
    if i in to_delete:
        continue
    version1.append(token)
version2 = []
to_delete = np.random.randint (0, len(tokens), int(0.3*len(tokens)))
for i, token in enumerate(tokens):
    if i in to_delete:
        continue
    version2.append(token)

## Test

In [12]:
print (len(longest_subsequence (version1, version2)))

400


## Narzędzie diff

Działa analogicznie do `longest_subsequence`, jednakże zapamiętuje przejścia w górę lub w lewo czyli miejsca gdzie w odległości edycyjnej usuwamy lub dodajemy element, są to elementy nie zgadzające się w obu tekstach. Tworzy opis lub zwraca tablicę z krotkami.

In [13]:
def diff (sequence1, sequence2, mode = 0):
    arr = levenshtein_distance (sequence1, sequence2, lss_delta)
    diff_line = [] #(value, file(1 or 2), line)
    i = len (sequence1)
    j = len (sequence2)
    while True:
        v = arr[i, j, 1]
        if v == 1:
            i -= 1
            diff_line.append((sequence1[i], 1, i))
        elif v == 2:
            j -= 1
            diff_line.append((sequence2[j], 2, j))
        elif v == 3:
            i -= 1
            j -= 1
        else:
            break
    diff_line.reverse()
    if mode:
        for text, file, line in diff_line:
            if isinstance (text, Token):
                text = text.text
            print (text + '\n', "-> linia", line, "w pliku", file)
    else:
        return diff_line

In [14]:
diff (version1, version2, 1)

Romeo i Julia
 -> linia 1 w pliku 2
tłum. Józef Paszkowski
 -> linia 2 w pliku 2

 -> linia 2 w pliku 1

 -> linia 6 w pliku 1
OSOBY:
 -> linia 7 w pliku 1
 * ESKALUS — książę panujący w Weronie
 -> linia 8 w pliku 1
 * STARZEC — stryjeczny brat Kapuleta
 -> linia 11 w pliku 1
 * ROMEO — syn Montekiego
 -> linia 12 w pliku 1
 * MERKUCJO — krewny księcia
 -> linia 13 w pliku 1
 * PIOTR
 -> linia 23 w pliku 1
 * PANI KAPULET — małżonka Kapuleta
 -> linia 25 w pliku 1
 * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
 -> linia 28 w pliku 1
Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.
 -> linia 32 w pliku 1

 -> linia 33 w pliku 1

 -> linia 34 w pliku 1

 -> linia 35 w pliku 1
PROLOG
 -> linia 24 w pliku 2

 -> linia 37 w pliku 1
Tam, gdzie się rzecz ta rozgrywa, w Weronie,
 -> linia 38 w pliku 1
Dwa rody, zacne jednako i sławne —
 -> linia 26 w pliku 2
Do nowej zbrodni pc